In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [2]:
spark = SparkSession \
        .builder \
        .appName("LocalApp") \
        .getOrCreate()

22/12/14 18:25:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
lineRDD = spark.sparkContext.textFile("../ds/wikiOfSpark.txt")
cleanWordRDD = lineRDD.flatMap(lambda line: line.split(" ")).filter(lambda word: word != "")

groupByKey

In [4]:
kvRDD = cleanWordRDD.map(lambda w:(w,w))
words = kvRDD.groupByKey()

words.take(3)

[('Apache', <pyspark.resultiterable.ResultIterable at 0x7fb6695b0f10>),
 ('Spark', <pyspark.resultiterable.ResultIterable at 0x7fb6695b0fd0>),
 ('Wikipedia,', <pyspark.resultiterable.ResultIterable at 0x7fb6695b0a30>)]

reduceByKey

In [5]:
# 把RDD元素转换为（Key，Value）的形式
kvRDD = cleanWordRDD.map(lambda word: (word, 1))
 
# 按照单词做分组计数
wordCounts = kvRDD.reduceByKey(lambda x, y: x + y)

wordCounts.take(3)

[('Apache', 34), ('Spark', 63), ('Wikipedia,', 1)]

In [6]:
import random
import math


# 显示定义提取最大值的聚合函数f
def f(x, y):
    return max(x, y)


kvRDD = cleanWordRDD.map(lambda word: (word, random.randint(1, 1000)))

# 按照单词提取最大值
wordCounts = kvRDD.reduceByKey(f)

wordCounts.take(3)

[('Apache', 994), ('Spark', 1000), ('Wikipedia,', 821)]

aggregateByKey

In [7]:
# 显示定义 Map 阶段聚合函数 f1
def f1(x, y):
    return x + y


# 显示定义 Reduce 阶段聚合函数 f2
def f2(x, y):
    return max(x, y)


kvRDD = cleanWordRDD.map(lambda w: (w, 1))

# 调用 aggregateByKey，实现先加和、再求最大值
wordCounts = kvRDD.aggregateByKey(0, seqFunc=f1, combFunc=f2)
print(wordCounts.take(3))

# 下面用 aggregateByKey 实现 reduceByKey 的功能
wordCounts = kvRDD.aggregateByKey(0, seqFunc=f1, combFunc=f1)

wordCounts.take(3)

[('Apache', 23), ('Spark', 39), ('Wikipedia,', 1)]


[('Apache', 34), ('Spark', 63), ('Wikipedia,', 1)]

sortByKey


In [8]:
sortedRDD = wordCounts.sortByKey()
# descending
# sortedRDD = wordCounts.sortByKey(False)

sortedRDD.take(3)

[('"))', 1), ('"12".', 1), ('"4', 1)]

In [9]:
# sortBy value

sortedRDD = wordCounts.sortBy(lambda x: x[1], False)

sortedRDD.take(3)

[('the', 67), ('Spark', 63), ('a', 54)]